In [ ]:
import csv
import pandas as pd
import sys
import numpy as np
from transliterate import translit, get_available_language_codes

In [ ]:
def delete_j(string):
    newstring = ""
    for i in range(0, len(string)):
        if string[i] == 'J':
            newstring += 'Y'
        else:
            newstring += string[i]
    return newstring

In [ ]:
def edit(string):
    splitted = string.split(' ')
    splitted[0] = translit(splitted[0], 'ru', reversed=True).upper()
    name1 = splitted[1]
    name2 = splitted[2]
    splitted[:] = (value for value in splitted if (value != '' and value != name1 and value != name2))
    name1 = translit(name1, 'ru', reversed=True).upper()
    if len(splitted[13]) == 1:
        splitted.insert(13, "N/A")
    flight_sh = splitted[6]
    if flight_sh[-1] == 'O':
        splitted[6] = flight_sh[:-2]
        splitted.insert(7, "NO")
    else:
        splitted[6] = flight_sh[:-3]
        splitted.insert(7, "YES")
    if len(splitted) == 17:
        for i in range(0, 3):
            splitted.append("N/A")
        splitted.insert(1, name1)
        splitted[0] = delete_j(splitted[0])
        splitted[1] = delete_j(splitted[1])
        return ",".join(splitted)
    if len(splitted[17]) != 1:
        if splitted[17] == "Assistance":
            splitted[17] += ' ' + splitted[18]
            del splitted[18]
        elif splitted[17] == "Travel":
            splitted[17] += ' ' + splitted[18] + ' ' + splitted[19]
            del splitted[18:20]
        else: splitted.insert(17, "N/A")
    if len(splitted) == 18:
        for i in range(0, 2):
            splitted.append("N/A")
        splitted.insert(1, name1)
        splitted[0] = delete_j(splitted[0])
        splitted[1] = delete_j(splitted[1])
        return ",".join(splitted)
    if splitted[18][2] != '#':
        splitted.insert(18, "N/A")
    else:
        splitted[18] += ' ' + splitted[19]
        del splitted[19]
    
    if len(splitted) == 19:
        splitted.append("N/A")
    splitted.insert(1, name1)
    
    splitted[0] = delete_j(splitted[0])
    splitted[1] = delete_j(splitted[1])
        
    return ",".join(splitted)

In [ ]:
tsv = open("Sirena-export-fixed.tab", 'r')
text = tsv.read()
lines = text.split('\n')
output = open("Sirena_edited.csv", 'w')
output.write("PaxLastName,PaxFirstName,PaxBirthDate,DepartDate,DepartTime,ArrivalDate,ArrivalTime,Flight,CodeSh,From,Dest,Code,e-Ticket,TravelDoc,Seat,Meal,TrvCls,Fare,Baggage,PaxAdditionalInfo,AgentInfo\n")
current = 1;
total = len(lines)
print(total)
for line in lines:
    if len(edited_line) and current != total:
        edited_line = edit(line)
        edited_line += "\n"
        output.write(edited_line)
    current += 1
tsv.close()
output.close()

In [ ]:
def norm(string):   
    if type(string) != str:
        return string
    string = string.upper()
    place_yi = string.find('YI')
    if place_yi != -1:
        string = string[:place_yi] + 'YY' + string[place_yi+2:]
        
    place_tc = string.find('TC')
    if place_tc != -1:
        string = string[:place_tc] + 'TS' + string[place_tc+2:]
        
    place_io = string.find('IO')
    if place_io != -1:
        string = string[:place_io] + 'YO' + string[place_io+2:]
        
    place_ii = string.find('II')
    if place_ii != -1:
        string = string[:place_ii] + 'IY' + string[place_ii+2:]
    
    place_x = string.find('X')
    if place_x != -1:
        string = string[:place_x] + 'KS' + string[place_x+1:]
    
    place_ia = string.find('IA')
    if place_ia != -1:
        string = string[:place_ia] + 'YA' + string[place_ia+2:]
    
    place_ia = string.find('AI')
    if place_ia != -1:
        string = string[:place_ia] + 'AY' + string[place_ia+2:]
    
    place_iu = string.find('IU')
    if place_iu != -1:
        string = string[:place_iu] + 'YU' + string[place_iu+2:]
        
    place_iu = string.find('UI')
    if place_iu != -1:
        string = string[:place_iu] + 'UY' + string[place_iu+2:]
    
    place_iu = string.find('EI')
    if place_iu != -1:
        string = string[:place_iu] + 'EY' + string[place_iu+2:]

    place_ump = string.find('\'')
    if place_ump != -1:
        string = string[:place_ump] + string[place_ump+1:]

    return string

In [ ]:
#загрузили всё
sirena_df = pd.read_csv("Sirena_edited.csv")
Senya_df = pd.read_csv("PointzAggregator.csv")
Maxon_df = pd.read_csv("FrequentFlyerForum-Profiles.csv")
excel_df = pd.read_csv('excel_df_correct_names.csv')
Boarding_df = pd.read_csv("BoardingData.csv", sep=';')

#пофиксили все имена
sirena_df['PaxLastName'] = sirena_df['PaxLastName'].apply(norm)
sirena_df['PaxFirstName'] = sirena_df['PaxFirstName'].apply(norm)

Senya_df['LastName'] = Senya_df['LastName'].apply(norm)
Senya_df['Name'] = Senya_df['Name'].apply(norm)

excel_df['FirstName'] = excel_df['FirstName'].apply(norm)
excel_df['LastName'] = excel_df['LastName'].apply(norm)

Boarding_df['PassengerFirstName'] = Boarding_df['PassengerFirstName'].apply(norm)
Boarding_df['PassengerLastName'] = Boarding_df['PassengerLastName'].apply(norm)

In [ ]:
Maxon_df['FirstName'] = Maxon_df['FirstName'].apply(norm)
Maxon_df['LastName'] = Maxon_df['LastName'].apply(norm)

In [ ]:
#поменяли названия колонок
sirena_df.rename(columns={'PaxLastName': 'LastName', 'PaxFirstName': 'FirstName'}, inplace=True)
Senya_df.rename(columns={'Name': 'FirstName'}, inplace=True)
excel_df.rename(columns={'FlightNumber': 'Flight'}, inplace=True)
Boarding_df.rename(columns={'PassengerLastName': 'LastName', 'PassengerFirstName': 'FirstName', 'FlightNumber': 'Flight'}, inplace=True)


In [ ]:
sirena_df.to_csv('maxrudik_file.csv')
Senya_df.to_csv('senya_file.csv')
excel_df.to_csv('ecxel_file.csv')
Boarding_df.to_csv('prev_csv_file.csv', sep=',')
Maxon_df.to_csv('maxon_file.csv')

In [ ]:
sirena_df.head(20)

In [ ]:
Senya_df.head(20)

In [ ]:
excel_df

In [ ]:
Maxon_df.head(300)

In [ ]:
excel_df.head(20)

In [ ]:
Boarding_df.head(20)

In [ ]:
#maxrudik_df.loc[maxrudik_df['Seat'].isna()]
maxrudik_df.head(10)
maxrudik_df.to_csv('maxrudik_file_temp.csv')

In [ ]:
maxrudik_df = pd.read_csv("normalized_csv/maxrudik_file.csv")

In [ ]:
#Senya_df.rename(columns={'Name': 'FirstName'}, inplace=True)
maxrudik_df.rename(columns={"BirthDate": "PassengerBirthDate",'DepartDate': 'FlightDate','CodeSh': 'CodeShare'}, inplace=True)
maxrudik_df.head(5)
maxrudik_df.to_csv("normalized_csv/maxrudik_file.csv")

In [ ]:
#maxrudik_df = pd.read('maxrudik_file.csv')
excel_df = pd.read_csv("normalized_csv/ecxel_file.csv")
#Prev_df = pd.read_csv("normalized_csv/prev_csv_file.csv")
#Senya_df = pd.read_csv("normalized_csv/senya_file.csv")
#Maxon_df = pd.read_csv("normalized_csv/maxon_file.csv")

In [ ]:
excel_df.rename(columns={'Date': 'FlightDate'}, inplace=True)
Senya_df.rename(columns={'Date': 'FlightDate'}, inplace=True)
Maxon_df.rename(columns={'Date': 'FlightDate'}, inplace=True)
 
Prev_df.rename(columns={'FlightTime': 'DepartTime','PassengerSex': 'Sex'}, inplace=True)
excel_df.rename(columns={'Time': 'DepartTime','Gender':'Sex'}, inplace=True)

Senya_df.rename(columns={'Code': 'Flight'}, inplace=True)

Maxon_df.rename(columns={'Codeshare': 'CodeShare'}, inplace=True)

In [ ]:
def norm_sex(string):
    if string == 'F':
        return "Female"
    else:
        return "Male"

In [ ]:
def norm_country(string):
    if type(string) != str:
        return string
    ans = string
    if string[:6] == "United":
        ans = "United states"
    if string[:7] == "Russian":
        ans = "Russian"
    if string[:5] == "Turks":
        ans = "Turks and Caicos"
    return ans.lower()

def norm_city(string):
    if type(string) != str:
        return string
    ans = string
    
    return ans.lower()

In [ ]:
#excel_df['Gender'] = excel_df['Gender'].apply(norm_sex)
#excel_df['ArrivalCity'] = excel_df['ArrivalCity'].apply(norm_city)
#excel_df['DepartureCity'] = excel_df['DepartureCity'].apply(norm_city)
#excel_df['DepartureCountry'] = excel_df['DepartureCountry'].apply(norm_country)
#excel_df['ArrivalCountry'] = excel_df['ArrivalCountry'].apply(norm_country)

#excel_df.loc[excel_df['DepartureCountry'].isna()]
excel_df

In [ ]:
#Maxon_df['ArrivalCity'] = Maxon_df['ArrivalCity'].apply(norm_city)
#Maxon_df['DepartureCity'] = Maxon_df['DepartureCity'].apply(norm_city)

In [ ]:
def norm_airport(string):
    if type(string) != str:
        return string
    else:
        return string.upper()

In [ ]:
#maxrudik_df.rename(columns={'From': 'DepartureAirport', 'Dest': 'ArrivalDate'}, inplace=True)
maxrudik_df = pd.read_csv('normalized_temp/maxrudik_file_temp.csv')
maxrudik_df["CodeShare"] = maxrudik_df["CodeShare"].apply(norm_ch)
#maxrudik_df["CodeShare"] = maxrudik_df["CodeShare"].apply(norm_ch)
#maxrudik_df["CodeShare"]

In [ ]:
def norm_ch(string):
    if string == False:
        return "Own"
    if string == "NO":
        return "Own"
    if string == True:
        return "Operated"
    if string == "YES":
        return "Operated"

In [ ]:
def norm_bonus(string):
    if string[0] == "F":
        return "FB"
    if string[0] == "K":
        return "KE"
    if string[0] == "D":
        return "DT"
    if string[0] == "A":
        return "SU"

In [215]:
maxrudik_df.head(5)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LastName,FirstName,PassengerBirthDate,FlightDate,DepartTime,ArrivalDate,ArrivalTime,...,Code,e-Ticket,TravelDoc,Seat,Meal,TrvCls,Fare,Baggage,PaxAdditionalInfo,AgentInfo
0,0,0,0,OZEROV,ILDAR,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,...,ZBQSPY7360415302044672,9375,53270,NaN,NaN,J,JGRPGN0PC,S,FF#SU 38116280,Go2See
1,1,1,1,KOLOSOV,SAMIR,NaN,2017-12-27,02:15,2017-12-27,04:40,...,MHPBBX7398421117936516,2244,645520,NaN,KSML,Y,YRSTUQ,9,FF#FB 284903754,NaN
2,2,2,2,IGNATOVA,SNEZHANA,NaN,2017-09-19,06:40,2017-09-19,07:45,...,REDLVB5174973140468001,8115,961316,NaN,NaN,Y,YSTNJL,F,NaN,KupiBilet
3,3,3,3,ZHAROV,PLATON,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,...,NSJNGQ5274206497242737,98,6865148,NaN,NaN,J,JFLXLS0PC,#,FF#FB 884556993,Travelgenio
4,4,4,4,NIKOLSKIY,NIKOLAY,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,...,VWNYGF6247422701565929,4396,926588,NaN,NaN,Y,YFLXPG,NaN,FF#SU 183142068,OZON.travel


In [216]:
excel_df.head(5)

,Unnamed: 0,Unnamed: 0.1,Gender,Flight,Date,Time,DepartureCountry,DepartureCity,DepartureAirport,ArrivalCountry,ArrivalCity,ArrivalAirport,AirlineName,LoyaltyStatus,Name,FirstName,LastName
0,0,0,Female,SU5436,2017-01-01,11:25,russia,vladivostok,VVO,french polynesia,seoul,REA,aeroflot,NaN,lidiya zhdanova,LIDIYA,ZHDANOVA
1,1,1,Male,SU5436,2017-01-01,11:25,russia,vladivostok,VVO,french polynesia,seoul,REA,aeroflot,NaN,sokolov sviatoslav,SVYATOSLAV,SOKOLOV
2,2,2,Male,SU5436,2017-01-01,11:25,russia,vladivostok,VVO,french polynesia,seoul,REA,aeroflot,NaN,sukhanov sviatoslav,SVYATOSLAV,SUKHANOV
3,3,3,Male,SU5436,2017-01-01,11:25,russia,vladivostok,VVO,french polynesia,seoul,REA,aeroflot,NaN,denis zelenin,DENIS,ZELENIN
4,4,4,Female,SU5436,2017-01-01,11:25,russia,vladivostok,VVO,french polynesia,seoul,REA,aeroflot,NaN,irina ostrovskaya,IRINA,OSTROVSKAYA


In [217]:
Prev_df.head(5)

,Unnamed: 0,FirstName,PassengerSecondName,LastName,Sex,PassengerBirthDate,PassengerDocument,BookingCode,TicketNumber,Baggage,FlightDate,DepartTime,Flight,CodeShare,ArrivalCity
0,0,SAVELIY,VIKTOROVICH,RUSANOV,Male,03/10/1983,2879 096860,FRNINO,6625956945991971,Transit,2017-03-22,06:05,SU1369,Own,moscow
1,1,LEV,MARKOVICH,ISAEV,Male,12/13/1975,1788 173211,Not presented,1643715499224676,Registered,2017-03-18,22:10,SU1180,Own,volgograd
2,2,NIKOLAY,I.,NIKOLSKIY,Male,12/26/1990,4396 926588,VWNYGF,6247422701565929,Transit,2017-03-18,22:10,SU1180,Own,volgograd
3,3,ANATOLIY,PETROVICH,SHILOV,Male,05/24/1997,2595 919752,WQFFUE,Not presented,Registered,2017-03-18,22:10,SU1180,Own,volgograd
4,4,MIROSLAVA,VIACHESLAVOVNA,SEMENOVA,Female,01/31/1976,6775 516990,Not presented,Not presented,Registered,2017-03-12,11:45,SU6284,Own,moscow


In [218]:
Senya_df.head(5)

,Unnamed: 0,UID,FirstName,LastName,CardType,CardNumber,BonusProgramm,Activities,Flight,FlightDate,DepartureAirport,ArrivalAirport,Fare
0,0,613142142,YAROMIR,ZVEREV,Airlines,FB 171388778,FB,Airlines,KE827,2017-08-06,REA,SZX,YGRPZT
1,1,613142142,YAROMIR,ZVEREV,Airlines,FB 171388778,FB,Airlines,MU9706,2017-10-26,PEK,BSD,YSTNYV
2,2,103197717,VITALINA,KOROVINA,Airlines,KE 696768759,KE,Airlines,DL5058,2017-09-11,CHA,ATL,YRSTKY
3,3,103197717,VITALINA,KOROVINA,Airlines,KE 696768759,KE,Airlines,KE1,2017-04-01,REA,HNL,PRSTRU
4,4,103197717,VITALINA,KOROVINA,Airlines,KE 696768759,KE,Airlines,DL837,2017-09-13,ATL,HNL,YGRPNG


In [219]:
Maxon_df.head(5)

,Unnamed: 0,FirstName,LastName,NickName,Sex,Status,BonusProgramm,Number,FlightDate,CodeShare,Flight,ArrivalCity,ArrivalAirport,ArrivalCountry,DepartureCity,DepartureAirport,DepartureCountry,ArrivaCountry,ArrivaCity
0,0,NaN,NaN,FrequentFlyer0,Male,Basic,KE,889215424,2017-03-04,Own,DL3377,pittsburgh pa,PIT,united states,boston ma,BOS,united states,united states,pittsburgh pa
1,1,NaN,NaN,FrequentFlyer0,Male,Elite+,DT,484697244,2017-03-04,Own,DL3377,pittsburgh pa,PIT,united states,boston ma,BOS,united states,united states,pittsburgh pa
2,2,NaN,NaN,FrequentFlyer0,Male,Basic,KE,889215424,2017-07-21,Own,DL3630,detroit mi,DTW,united states,green bay wi,GRB,united states,united states,detroit mi
3,3,NaN,NaN,FrequentFlyer0,Male,Elite+,DT,484697244,2017-07-21,Own,DL3630,detroit mi,DTW,united states,green bay wi,GRB,united states,united states,detroit mi
4,4,NaN,NaN,FrequentFlyer0,Male,Basic,KE,889215424,2017-03-22,Own,DL5361,new york ny,LGA,united states,pittsburgh pa,PIT,united states,united states,new york ny


In [ ]:
#maxrudik_with_index = maxrudik_df.set_index(["LastName", "FirstName", "Flight"])
#excel_with_index = excel_df.set_index(["LastName", "FirstName", "Flight"])

#repeat_index = maxrudik_with_index.index & excel_with_index.index
#uniqie_maxrudik_with_index = maxrudik_with_index.drop(repeat_index, axis=0)
#max_excel = pd.concat((uniqie_maxrudik_with_index, excel_with_index))
max_excel

In [230]:
prev_with_index = Prev_df.set_index(["LastName", "FirstName", "Flight"])

repeat_index = max_excel.index & prev_with_index.index
uniqie_prev_with_index = prev_with_index.drop(repeat_index, axis=0)
max_excel_prev = pd.concat((max_excel, uniqie_prev_with_index))
max_excel_prev

,,,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,PassengerBirthDate,FlightDate,DepartTime,ArrivalDate,ArrivalTime,CodeShare,DepartureAirport,...,ArrivalCity,ArrivalAirport,AirlineName,LoyaltyStatus,Name,PassengerSecondName,Sex,PassengerDocument,BookingCode,TicketNumber
LastName,FirstName,Flight,,,,,,,,,,,,,,,,,,,,,
OZEROV,ILDAR,SU1306,0,0.0,0.0,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,Own,SVO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KOLOSOV,SAMIR,SU1323,1,1.0,1.0,NaN,2017-12-27,02:15,2017-12-27,04:40,Own,MMK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IGNATOVA,SNEZHANA,SU1481,2,2.0,2.0,NaN,2017-09-19,06:40,2017-09-19,07:45,Own,KJA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZHAROV,PLATON,SU1180,3,3.0,3.0,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,Own,SVO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIKOLSKIY,NIKOLAY,SU1180,4,4.0,4.0,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,Own,SVO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PASTUKHOV,ELISEY,SU1181,155137,NaN,NaN,02/06/1975,2017-02-14,03:10,NaN,NaN,Own,NaN,...,moscow,NaN,NaN,NaN,NaN,IGNATOVICH,Male,5860 040208,Not presented,3425220683742061
GOROKHOV,BORIS,SU1181,155139,NaN,NaN,10/28/1970,2017-02-14,03:10,NaN,NaN,Own,NaN,...,moscow,NaN,NaN,NaN,NaN,S.,Male,7351 760618,Not presented,1569746161308219
TIMOFEEV,ARTEMIY,SU1181,155140,NaN,NaN,04/19/1981,2017-02-14,03:10,NaN,NaN,Own,NaN,...,moscow,NaN,NaN,NaN,NaN,I.,Male,4553 277857,Not presented,9914441929917761


In [234]:
senya_with_index = Senya_df.set_index(["LastName", "FirstName", "Flight"])

repeat_index = max_excel_prev.index & senya_with_index.index
unique_senya_with_index = senya_with_index.drop(repeat_index, axis=0)
max_excel_prev_senya = pd.concat((max_excel_prev, unique_senya_with_index))
max_excel_prev_senya

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  \
LastName  FirstName Flight                                             
OZEROV    ILDAR     SU1306           0           0.0             0.0   
KOLOSOV   SAMIR     SU1323           1           1.0             1.0   
IGNATOVA  SNEZHANA  SU1481           2           2.0             2.0   
ZHAROV    PLATON    SU1180           3           3.0             3.0   
NIKOLSKIY NIKOLAY   SU1180           4           4.0             4.0   
...                                ...           ...             ...   
KAZAKOV   EMIL      SU6402      436085           NaN             NaN   
                    AR1142      436086           NaN             NaN   
                    AR1143      436087           NaN             NaN   
KONONOVA  ULYANA    GA502       436088           NaN             NaN   
                    GA88        436089           NaN             NaN   

                           PassengerBirthDate  FlightDate DepartTime  \
LastName  FirstName Flight                                             
OZEROV    ILDAR     SU1306         1999-05-15  2017-05-30      00:05   
KOLOSOV   SAMIR     SU1323                NaN  2017-12-27      02:15   
IGNATOVA  SNEZHANA  SU1481                NaN  2017-09-19      06:40   
ZHAROV    PLATON    SU1180         1999-05-02  2017-03-18      22:10   
NIKOLSKIY NIKOLAY   SU1180         1990-12-26  2017-03-18      22:10   
...                                       ...         ...        ...   
KAZAKOV   EMIL      SU6402                NaN  2017-03-01        NaN   
                    AR1142                NaN  2017-12-14        NaN   
                    AR1143                NaN  2017-09-16        NaN   
KONONOVA  ULYANA    GA502                 NaN  2017-05-12        NaN   
                    GA88                  NaN  2017-08-02        NaN   

                           ArrivalDate ArrivalTime CodeShare DepartureAirport  \
LastName  FirstName Flight                                                      
OZEROV    ILDAR     SU1306  2017-05-30       08:05       Own              SVO   
KOLOSOV   SAMIR     SU1323  2017-12-27       04:40       Own              MMK   
IGNATOVA  SNEZHANA  SU1481  2017-09-19       07:45       Own              KJA   
ZHAROV    PLATON    SU1180  2017-03-19       01:05       Own              SVO   
NIKOLSKIY NIKOLAY   SU1180  2017-03-19       01:05       Own              SVO   
...                                ...         ...       ...              ...   
KAZAKOV   EMIL      SU6402         NaN         NaN       NaN              SVX   
                    AR1142         NaN         NaN       NaN              EZE   
                    AR1143         NaN         NaN       NaN              FCO   
KONONOVA  ULYANA    GA502          NaN         NaN       NaN              CGK   
                    GA88           NaN         NaN       NaN              CGK   

                            ... PassengerSecondName  Sex  PassengerDocument  \
LastName  FirstName Flight  ...                                               
OZEROV    ILDAR     SU1306  ...                 NaN  NaN                NaN   
KOLOSOV   SAMIR     SU1323  ...                 NaN  NaN                NaN   
IGNATOVA  SNEZHANA  SU1481  ...                 NaN  NaN                NaN   
ZHAROV    PLATON    SU1180  ...                 NaN  NaN                NaN   
NIKOLSKIY NIKOLAY   SU1180  ...                 NaN  NaN                NaN   
...                         ...                 ...  ...                ...   
KAZAKOV   EMIL      SU6402  ...                 NaN  NaN                NaN   
                    AR1142  ...                 NaN  NaN                NaN   
                    AR1143  ...                 NaN  NaN                NaN   
KONONOVA  ULYANA    GA502   ...                 NaN  NaN                NaN   
                    GA88    ...                 NaN  NaN                NaN   

                            BookingCode  TicketNumber          UID  Card

In [237]:
maxon_with_index = Maxon_df.set_index(["LastName", "FirstName", "Flight"])

repeat_index = max_excel_prev_senya.index & maxon_with_index.index
unique_maxon_with_index = maxon_with_index.drop(repeat_index, axis=0)
all_concated = pd.concat((max_excel_prev_senya, unique_maxon_with_index))
all_concated

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  \
LastName  FirstName Flight                                             
OZEROV    ILDAR     SU1306           0           0.0             0.0   
KOLOSOV   SAMIR     SU1323           1           1.0             1.0   
IGNATOVA  SNEZHANA  SU1481           2           2.0             2.0   
ZHAROV    PLATON    SU1180           3           3.0             3.0   
NIKOLSKIY NIKOLAY   SU1180           4           4.0             4.0   
...                                ...           ...             ...   
NaN       NaN       SU1185     1747126           NaN             NaN   
                    SU2033     1747127           NaN             NaN   
                    SU2033     1747128           NaN             NaN   
                    SU2033     1747129           NaN             NaN   
                    SU2033     1747130           NaN             NaN   

                           PassengerBirthDate  FlightDate DepartTime  \
LastName  FirstName Flight                                             
OZEROV    ILDAR     SU1306         1999-05-15  2017-05-30      00:05   
KOLOSOV   SAMIR     SU1323                NaN  2017-12-27      02:15   
IGNATOVA  SNEZHANA  SU1481                NaN  2017-09-19      06:40   
ZHAROV    PLATON    SU1180         1999-05-02  2017-03-18      22:10   
NIKOLSKIY NIKOLAY   SU1180         1990-12-26  2017-03-18      22:10   
...                                       ...         ...        ...   
NaN       NaN       SU1185                NaN  2017-06-26        NaN   
                    SU2033                NaN  2017-05-07        NaN   
                    SU2033                NaN  2017-05-07        NaN   
                    SU2033                NaN  2017-05-07        NaN   
                    SU2033                NaN  2017-05-07        NaN   

                           ArrivalDate ArrivalTime CodeShare DepartureAirport  \
LastName  FirstName Flight                                                      
OZEROV    ILDAR     SU1306  2017-05-30       08:05       Own              SVO   
KOLOSOV   SAMIR     SU1323  2017-12-27       04:40       Own              MMK   
IGNATOVA  SNEZHANA  SU1481  2017-09-19       07:45       Own              KJA   
ZHAROV    PLATON    SU1180  2017-03-19       01:05       Own              SVO   
NIKOLSKIY NIKOLAY   SU1180  2017-03-19       01:05       Own              SVO   
...                                ...         ...       ...              ...   
NaN       NaN       SU1185         NaN         NaN       Own              VOG   
                    SU2033         NaN         NaN       Own              BUD   
                    SU2033         NaN         NaN       Own              BUD   
                    SU2033         NaN         NaN       Own              BUD   
                    SU2033         NaN         NaN       Own              BUD   

                            ... UID CardType  CardNumber  BonusProgramm  \
LastName  FirstName Flight  ...                                           
OZEROV    ILDAR     SU1306  ... NaN      NaN         NaN            NaN   
KOLOSOV   SAMIR     SU1323  ... NaN      NaN         NaN            NaN   
IGNATOVA  SNEZHANA  SU1481  ... NaN      NaN         NaN            NaN   
ZHAROV    PLATON    SU1180  ... NaN      NaN         NaN            NaN   
NIKOLSKIY NIKOLAY   SU1180  ... NaN      NaN         NaN            NaN   
...                         ...  ..      ...         ...            ...   
NaN       NaN       SU1185  ... NaN      NaN         NaN             DT   
                    SU2033  ... NaN      NaN         NaN             FB   
                    SU2033  ... NaN      NaN         NaN             KE   
                    SU2033  ... NaN      NaN         NaN             SU   
                    SU2033  ... NaN      NaN         NaN             DT   

                            Activities            NickName  Status  \
LastName  FirstName Flight                            

In [238]:
all_concated.to_csv('all_concated.csv')

In [240]:
all_concated

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  \
LastName  FirstName Flight                                             
OZEROV    ILDAR     SU1306           0           0.0             0.0   
KOLOSOV   SAMIR     SU1323           1           1.0             1.0   
IGNATOVA  SNEZHANA  SU1481           2           2.0             2.0   
ZHAROV    PLATON    SU1180           3           3.0             3.0   
NIKOLSKIY NIKOLAY   SU1180           4           4.0             4.0   
...                                ...           ...             ...   
NaN       NaN       SU1185     1747126           NaN             NaN   
                    SU2033     1747127           NaN             NaN   
                    SU2033     1747128           NaN             NaN   
                    SU2033     1747129           NaN             NaN   
                    SU2033     1747130           NaN             NaN   

                           PassengerBirthDate  FlightDate DepartTime  \
LastName  FirstName Flight                                             
OZEROV    ILDAR     SU1306         1999-05-15  2017-05-30      00:05   
KOLOSOV   SAMIR     SU1323                NaN  2017-12-27      02:15   
IGNATOVA  SNEZHANA  SU1481                NaN  2017-09-19      06:40   
ZHAROV    PLATON    SU1180         1999-05-02  2017-03-18      22:10   
NIKOLSKIY NIKOLAY   SU1180         1990-12-26  2017-03-18      22:10   
...                                       ...         ...        ...   
NaN       NaN       SU1185                NaN  2017-06-26        NaN   
                    SU2033                NaN  2017-05-07        NaN   
                    SU2033                NaN  2017-05-07        NaN   
                    SU2033                NaN  2017-05-07        NaN   
                    SU2033                NaN  2017-05-07        NaN   

                           ArrivalDate ArrivalTime CodeShare DepartureAirport  \
LastName  FirstName Flight                                                      
OZEROV    ILDAR     SU1306  2017-05-30       08:05       Own              SVO   
KOLOSOV   SAMIR     SU1323  2017-12-27       04:40       Own              MMK   
IGNATOVA  SNEZHANA  SU1481  2017-09-19       07:45       Own              KJA   
ZHAROV    PLATON    SU1180  2017-03-19       01:05       Own              SVO   
NIKOLSKIY NIKOLAY   SU1180  2017-03-19       01:05       Own              SVO   
...                                ...         ...       ...              ...   
NaN       NaN       SU1185         NaN         NaN       Own              VOG   
                    SU2033         NaN         NaN       Own              BUD   
                    SU2033         NaN         NaN       Own              BUD   
                    SU2033         NaN         NaN       Own              BUD   
                    SU2033         NaN         NaN       Own              BUD   

                            ... UID CardType  CardNumber  BonusProgramm  \
LastName  FirstName Flight  ...                                           
OZEROV    ILDAR     SU1306  ... NaN      NaN         NaN            NaN   
KOLOSOV   SAMIR     SU1323  ... NaN      NaN         NaN            NaN   
IGNATOVA  SNEZHANA  SU1481  ... NaN      NaN         NaN            NaN   
ZHAROV    PLATON    SU1180  ... NaN      NaN         NaN            NaN   
NIKOLSKIY NIKOLAY   SU1180  ... NaN      NaN         NaN            NaN   
...                         ...  ..      ...         ...            ...   
NaN       NaN       SU1185  ... NaN      NaN         NaN             DT   
                    SU2033  ... NaN      NaN         NaN             FB   
                    SU2033  ... NaN      NaN         NaN             KE   
                    SU2033  ... NaN      NaN         NaN             SU   
                    SU2033  ... NaN      NaN         NaN             DT   

                            Activities            NickName  Status  \
LastName  FirstName Flight                            

In [243]:
#number_of_bd = all_concated.groupby(["LastName","FirstName"])['PassengerBirthDate'].count()
#number_of_bd.unique()


array([ 1,  0,  2,  7,  3,  4,  6,  5, 10,  9,  8, 11, 14, 13, 12, 18, 19,
       15, 16, 17, 20])

In [244]:
final_df = pd.read_csv('all_fixed.csv')

/home/max/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,33,34,35,36,38,39,40,41,42,44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [275]:
#final_df = final_df.set_index(["LastName", "FirstName", "Flight"])
#final_df_temp = final_df
#final_df_temp['unique_bd'] = final_df.groupby(["LastName","FirstName"])['PassengerBirthDate'].nunique()
#final_df_temp[final_df_temp['LastName'] == "OZEROV"]
final_df_temp.loc[final_df_temp["LastName"] == "OZEROV"]

KeyError: 'LastName'

In [ ]:
maxrudik_df.to_csv('normalized_temp/maxrudik_file_temp.csv')
excel_df.to_csv("normalized_temp/ecxel_file_temp.csv")
Prev_df.to_csv("normalized_temp/prev_csv_file_temp.csv")
Senya_df.to_csv("normalized_temp/senya_file_temp.csv")
Maxon_df.to_csv("normalized_temp/maxon_file_temp.csv")